# **Emotion Classifier**



## 1. Libraries

In [ ]:
# Installing libraries

!pip install datasets
!pip install transformers
!pip install nltk emoji==0.6.0
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.7 MB/s 
     |████████████████████████████████| 141 kB 51.7 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 212 kB 15.5 MB/s 
     |████████████████████████████████| 115 kB 53.3 MB/s 
     |████████████████████████████████| 596 kB 38.1 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.7 MB/s 
     |█████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import tensorflow as tf
import tensorflow_addons as tfa

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, AutoTokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict, defaultdict, Counter
import pandas as pd
import csv
from nltk.tokenize import TweetTokenizer

## Dataset

In [ ]:
# SemEval dataset
semeval = load_dataset('sem_eval_2018_task_1', 'subtask5.english')
data = semeval.data


label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

train_set = data["train"].to_pandas()
val_set = data["validation"].to_pandas()
test_set = data["test"].to_pandas()

train_set.drop('ID', inplace=True, axis=1)
val_set.drop('ID', inplace=True, axis=1)
test_set.drop('ID', inplace=True, axis=1)

frames = [train_set, val_set]
train_set = pd.concat(frames)
train_set = train_set.reset_index(drop=True)


print(train_set.shape, val_set.shape, test_set.shape)


test_set.head()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Dataset sem_eval_2018_task_1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

(7724, 12) (886, 12) (3259, 12)


,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,True,True,False,False,False,False,True,False,False,False,True
1,"Academy of Sciences, eschews the normally sobe...",False,False,True,False,False,False,False,False,False,False,False
2,I blew that opportunity -__- #mad,True,False,True,False,False,False,False,False,True,False,False
3,This time in 2 weeks I will be 30... 😥,False,False,False,False,True,False,False,False,True,False,False
4,#Deppression is real. Partners w/ #depressed p...,False,False,False,True,False,False,False,False,True,False,False


In [ ]:
# Normalise BERTweet datasets
from emoji import demojize
from nltk.tokenize import TweetTokenizer


tokenizer = TweetTokenizer()


def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())

In [ ]:
def normalise_dataset(dataset):
  for entry in dataset['Tweet']:
    dataset['Tweet'] = dataset['Tweet'].replace(entry, normalizeTweet(entry))


normalise_dataset(train_set)
normalise_dataset(val_set)
normalise_dataset(test_set)

In [ ]:
train_set.head(10)

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“ Worry is a down payment on a problem you may...,False,True,False,False,False,False,True,False,False,False,True
1,Whatever you decide to do make sure it makes y...,False,False,False,False,True,True,True,False,False,False,False
2,@USER it also helps that the majority of NFL c...,True,False,True,False,True,False,True,False,False,False,False
3,Accept the challenges so that you can literall...,False,False,False,False,True,False,True,False,False,False,False
4,My roommate : it 's okay that we can't spell b...,True,False,True,False,False,False,False,False,False,False,False
5,No but that 's so cute . Atsu was probably shy...,False,False,False,False,True,False,False,False,False,False,False
6,Do you think humans have the sense for recogni...,False,True,False,False,False,False,False,True,False,False,False
7,Rooneys fucking untouchable is n't he ? Been f...,True,False,True,False,False,False,False,False,False,False,False
8,it 's pretty depressing when u hit pan on ur f...,False,False,True,False,False,False,False,False,True,False,False
9,@USER but your pussy was weak from what I hear...,True,False,True,False,False,False,False,False,False,False,False


In [ ]:
#Lexicon normal words

wordList = defaultdict(list)
emotionList = defaultdict(list)
with open('/content/drive/MyDrive/NRC-Lexicon/NRC-Hashtag-Emotion-Lexicon-v0.2 3/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    headerRows = [i for i in range(0, 0)]
    for row in headerRows:
        next(reader)
    for word, emotion, present in reader:
        if int(present) == 1:
            #print(word)
            wordList[word].append(emotion)
            emotionList[emotion].append(word)

tt = TweetTokenizer()

def generate_emotion_count(string, tokenizer):
    emoCount = Counter()    
    for token in tt.tokenize(string):
        token = token.lower()
        emoCount += Counter(wordList[token])
    return emoCount

In [ ]:
#Lexicon hashtags

wordList_hash = defaultdict(list)
emotionList_hash = defaultdict(list)
with open('/content/drive/MyDrive/NRC-Lexicon/NRC-Hashtag-Emotion-Lexicon-v0.2 3/NRC-Hashtag-Emotion-Lexicon-v0.2.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    headerRows = [i for i in range(0, 0)]
    for row in headerRows:
      next(reader)
    for emotion, word, score in reader:
      wordList_hash[word].append({emotion:score})


def generate_emotion_count_hash(string, tokenizer):
    emo_count_hash = defaultdict(lambda: 0)
    for token in tt.tokenize(string):
        token = token.lower()
        for i in wordList_hash[token]:
          for j in i:
            emo_count_hash[j] += float(i.get(j))
          
    return emo_count_hash

In [ ]:
# # Lexicon Emojis
from nltk.tokenize import word_tokenize

emoji_df = pd.read_csv("/content/drive/MyDrive/NRC-Lexicon/NRC-Hashtag-Emotion-Lexicon-v0.2 3/Emoji_Lexicon.csv")
emoji_df.drop('id', inplace=True, axis=1)


for i, emoji in enumerate(emoji_df['emoji']):
  emoji_df['emoji'][i] = demojize(emoji)


emoji_dict = defaultdict(list)
emoji_list = emoji_df.values.tolist()

for i in emoji_list:
  emoji_dict[i[1]] = i[2:10]

In [ ]:
def generate_emoji_emotion(string, tokenizer):
    arr = [[0]*8]
    for token in string.split(" "):
      if emoji_dict[token] != []:
        arr.append(emoji_dict[token])
    return np.sum(arr, axis = 0)

In [ ]:
# Word Lexicon
emotionCounts_train = [generate_emotion_count(tweet, tt) for tweet in train_set['Tweet']]
emotionCounts_val = [generate_emotion_count(tweet, tt) for tweet in val_set['Tweet']]
emotionCounts_test = [generate_emotion_count(tweet, tt) for tweet in test_set['Tweet']]

# Hashtag Lexicon 
emotionCounts_hash_train = [generate_emotion_count_hash(tweet, tt) for tweet in train_set['Tweet']]
emotionCounts_hash_val = [generate_emotion_count_hash(tweet, tt) for tweet in val_set['Tweet']]
emotionCounts_hash_test = [generate_emotion_count_hash(tweet, tt) for tweet in test_set['Tweet']]

# Emoji Lexicon
emoji_train = [generate_emoji_emotion(tweet, tt) for tweet in train_set['Tweet']]
emoji_val = [generate_emoji_emotion(tweet, tt) for tweet in val_set['Tweet']]
emoji_test = [generate_emoji_emotion(tweet, tt) for tweet in test_set['Tweet']]

In [ ]:
# Word Lexicon
emotion_df_train = pd.DataFrame(emotionCounts_train, index=train_set['Tweet'].index)
emotion_df_train = emotion_df_train.fillna(0)

emotion_df_val = pd.DataFrame(emotionCounts_val, index=val_set['Tweet'].index)
emotion_df_val = emotion_df_val.fillna(0)

emotion_df_test = pd.DataFrame(emotionCounts_test, index=test_set['Tweet'].index)
emotion_df_test = emotion_df_test.fillna(0)

# Hashtag Lexicon
emotion_df_hash_train = pd.DataFrame(emotionCounts_hash_train, index=train_set['Tweet'].index)
emotion_df_hash_train = emotion_df_hash_train.fillna(0)

emotion_df_hash_val = pd.DataFrame(emotionCounts_hash_val, index=val_set['Tweet'].index)
emotion_df_hash_val = emotion_df_hash_val.fillna(0)

emotion_df_hash_test = pd.DataFrame(emotionCounts_hash_test, index=test_set['Tweet'].index)
emotion_df_hash_test = emotion_df_hash_test.fillna(0)

In [ ]:
# Word Lexicon
emotion_lexicon_train = emotion_df_train.to_numpy(dtype=np.float32).tolist()
emotion_lexicon_val = emotion_df_val.to_numpy(np.float32).tolist()
emotion_lexicon_test = emotion_df_test.to_numpy(np.float32).tolist()

# Hashtag Lexicon
emotion_lexicon_hash_train = emotion_df_hash_train.to_numpy(dtype=np.float32).tolist()
emotion_lexicon_hash_val = emotion_df_hash_val.to_numpy(dtype=np.float32).tolist()
emotion_lexicon_hash_test = emotion_df_hash_test.to_numpy(dtype=np.float32).tolist()

In [ ]:
print(emotion_lexicon_hash_train)

[[2.330031633377075, 3.8696787357330322, 2.480886459350586, 0.19202369451522827, 1.3785220384597778, 0.20613832771778107, 1.0391325950622559, 0.629096508026123], [0.2767409086227417, 0.7759236097335815, 2.201770067214966, 0.05872628465294838, 0.8128360509872437, 0.6820618510246277, 0.0, 0.0414547398686409], [0.0, 0.2632191479206085, 0.12227257341146469, 1.5442801713943481, 0.7660760283470154, 1.1720632314682007, 2.2132067680358887, 0.1790446639060974], [0.991278886795044, 3.4835798740386963, 1.232422947883606, 0.22838205099105835, 0.1595964878797531, 0.7331701517105103, 0.48208627104759216, 0.49946171045303345], [0.06487997621297836, 0.5449724197387695, 0.0, 0.0, 3.754728317260742, 1.265845537185669, 0.10038512945175171, 0.2767431437969208], [0.7024398446083069, 0.28475967049598694, 0.21590116620063782, 1.6123884916305542, 0.37693530321121216, 1.4610486030578613, 0.1697789430618286, 0.44174864888191223], [1.6453841924667358, 0.4930345416069031, 0.0, 0.05872628465294838, 0.4900877773761

In [ ]:
# Divide vectors by length of sentences

# Word Lexicon

for i, arr in enumerate(emotion_lexicon_train):
  emotion_lexicon_train[i] = [j/len(tt.tokenize(train_set['Tweet'][i])) for j in emotion_lexicon_train[i]]

for i, arr in enumerate(emotion_lexicon_val):
  emotion_lexicon_val[i] = [j/len(tt.tokenize(val_set['Tweet'][i])) for j in emotion_lexicon_val[i]]

for i, arr in enumerate(emotion_lexicon_test):
  emotion_lexicon_test[i] = [j/len(tt.tokenize(test_set['Tweet'][i])) for j in emotion_lexicon_test[i]]

# Hashtag Lexicon

for i, arr in enumerate(emotion_lexicon_hash_train):
  emotion_lexicon_hash_train[i] = [j/len(tt.tokenize(train_set['Tweet'][i])) for j in emotion_lexicon_hash_train[i]]

for i, arr in enumerate(emotion_lexicon_hash_val):
  emotion_lexicon_hash_val[i] = [j/len(tt.tokenize(val_set['Tweet'][i])) for j in emotion_lexicon_hash_val[i]]

for i, arr in enumerate(emotion_lexicon_hash_test):
  emotion_lexicon_hash_test[i] = [j/len(tt.tokenize(test_set['Tweet'][i])) for j in emotion_lexicon_hash_test[i]]

# Emoji 

for i, arr in enumerate(emoji_train):
  emoji_train[i] = [j/len(tt.tokenize(train_set['Tweet'][i])) for j in emoji_train[i]]

for i, arr in enumerate(emoji_val):
  emoji_val[i] = [j/len(tt.tokenize(val_set['Tweet'][i])) for j in emoji_val[i]]

for i, arr in enumerate(emoji_test):
  emoji_test[i] = [j/len(tt.tokenize(test_set['Tweet'][i])) for j in emoji_test[i]]

In [ ]:
print(emotion_lexicon_hash_train)

[[0.11095388679885869, 0.1842704161774535, 0.1181374548674199, 0.009143985110037896, 0.06564390638044657, 0.009816110811943123, 0.04948250512438491, 0.029956976631093007], [0.023061742811782332, 0.06466029926132309, 0.183480840670015, 0.0048938571563737, 0.06773633710933084, 0.05683848622597217, 0.0, 0.0034545617735531834], [0.0, 0.010123813866389192, 0.004702791173629885, 0.05939539105101029, 0.02946446282020992, 0.04507935606746798, 0.08512333848991838, 0.006886333110489635], [0.03965115651004707, 0.13934319663156858, 0.04929691557070433, 0.009135281748964252, 0.006383859752571048, 0.0293268059025228, 0.01928345082320432, 0.019978467977176808], [0.0036044431057637, 0.03027624524631927, 0.0, 0.0, 0.20859601179271398, 0.07032475334679589, 0.0055769517258642785, 0.015374619773850998], [0.03344951632217065, 0.013559984868960762, 0.010281008027808953, 0.07678040643677267, 0.017949299943803888, 0.06957374253797671, 0.008084711392167047, 0.021035649459736187], [0.13711535294238938, 0.041086

In [ ]:
tf.random.set_seed(1234)
#bert_model_name = "bert-base-uncased"
#bert_model_name = "bert-large-uncased"
#bert_model_name = "vinai/bertweet-base"
bert_model_name = "vinai/bertweet-large"

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
#tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")
MAX_LEN = 64

def tokenize_sentences(sentences, tokenizer, max_seq_len = 64):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens = True, max_length = max_seq_len)
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

train_input_ids = tokenize_sentences(train_set['Tweet'], tokenizer, MAX_LEN)
print(train_input_ids)
train_inputs = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_masks = create_attention_masks(train_inputs)

val_input_ids = tokenize_sentences(val_set['Tweet'], tokenizer, MAX_LEN)
val_inputs = pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_masks = create_attention_masks(val_inputs)

test_input_ids = tokenize_sentences(test_set['Tweet'], tokenizer, MAX_LEN)
test_inputs = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_masks = create_attention_masks(test_inputs)

train_labels =  train_set[label_cols].values
val_labels = val_set[label_cols]
test_labels = test_set[label_cols].values

  0%|          | 0/7724 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[0, 17, 48, 305, 17649, 16, 10, 159, 3207, 15, 10, 936, 47, 189, 393, 33, 128, 479, 12181, 11392, 479, 849, 25331, 38591, 849, 23240, 4128, 849, 605, 17649, 2], [0, 25216, 47, 2845, 7, 109, 146, 686, 24, 817, 47, 849, 27333, 479, 2], [0, 1039, 47955, 24, 67, 2607, 14, 5, 1647, 9, 1485, 4880, 16, 35335, 479, 993, 9, 1585, 384, 108, 9814, 128, 29, 310, 1765, 21, 26388, 2156, 27785, 849, 43070, 26063, 2], [0, 47127, 5, 2019, 98, 14, 47, 64, 5909, 190, 619, 5, 32749, 1258, 9, 1124, 479, 128, 111, 111, 1655, 208, 479, 24040, 4832, 16319, 1215, 9021, 35, 2], [0, 2387, 25537, 4832, 24, 128, 29, 8578, 14, 52, 64, 75, 8921, 142, 52, 33, 7241, 1975, 42892, 479, 849, 1334, 24786, 849, 9502, 8331, 4892, 4311, 2], [0, 3084, 53, 14, 128, 29, 98, 11962, 479, 83, 1872, 257, 21, 1153, 9152, 59, 2356, 137, 53, 20075, 1147, 69, 66, 1717, 13104, 2], [0, 8275, 47, 206, 5868, 33, 5, 1472, 13, 16257, 18049, 27833, 17487, 2], [0, 27110, 1264, 2459, 23523, 7587, 12746, 868, 16, 295, 75, 37, 17487, 30857, 2352

  0%|          | 0/886 [00:00<?, ?it/s]

  0%|          | 0/3259 [00:00<?, ?it/s]

In [ ]:
print(test_input_ids[3])
print(tokenizer.decode(test_input_ids[3]))

[0, 713, 86, 11, 132, 688, 38, 40, 28, 389, 1666, 4832, 29, 625, 1215, 4297, 1215, 5982, 36024, 1215, 9021, 35, 2]
<s>This time in 2 weeks I will be 30... :sad_but_relieved_face:</s>


In [ ]:
BATCH_SIZE = 16
NR_EPOCHS = 3

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    
    return dataset

train_dataset = create_dataset((train_inputs, train_masks, train_labels, emotion_lexicon_train, emotion_lexicon_hash_train, emoji_train), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((val_inputs, val_masks, val_labels, emotion_lexicon_val, emotion_lexicon_hash_val, emoji_val), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
test_dataset = create_dataset((test_inputs, test_masks, emotion_lexicon_test, emotion_lexicon_hash_test, emoji_test), batch_size=BATCH_SIZE, train=False, epochs=1)

## 3. BERT model


In [ ]:
from transformers import TFBertModel, TFAutoModel, AutoTokenizer, TFRobertaModel


class AutoClassifier(tf.keras.Model):
      def __init__(self, bert: TFRobertaModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = tf.keras.layers.Dense(num_classes, activation='sigmoid')
        self.pre_classifier = tf.keras.layers.Dense(2048, activation='relu')
      @tf.function
      def call(self, input_ids, lexicon, hash_lexicon, emoji, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        

        lex = tf.convert_to_tensor(lexicon)
        hash_lex = tf.convert_to_tensor(hash_lexicon)
        emoji_lex = tf.convert_to_tensor(emoji)
        emoji_lex = tf.cast(emoji_lex, lex.dtype)

        
        cls_output = tf.concat([outputs[1], lex, hash_lex, emoji_lex], 1)
        cls_output = self.pre_classifier(cls_output)
        cls_output = self.classifier(cls_output)
                
        return cls_output


model = AutoClassifier(TFAutoModel.from_pretrained(bert_model_name), len(label_cols))

11


Some layers from the model checkpoint at vinai/bertweet-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta/pooler/dense/kernel:0', 'roberta/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(train_inputs[:1])

model(train_inputs[:1], emotion_lexicon_train[:1], emotion_lexicon_hash_train[:1], emoji_train[:1], train_masks[:1]).numpy()
model.summary()

[[    0    17    48   305 17649    16    10   159  3207    15    10   936
     47   189   393    33   128   479 12181 11392   479   849 25331 38591
    849 23240  4128   849   605 17649     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
(1, 1024)
Tensor("Cast:0", shape=(1, 8), dtype=float32)
(1, 8)
(1, 1050)
(1, 1024)
Tensor("Cast:0", shape=(1, 8), dtype=float32)
(1, 8)
(1, 1050)
Model: "auto_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_roberta_model (TFRoberta  multiple                 355359744 
 Model)                                                          
                                                                 
 dense (Dense)               multiple                  22539     
                                   

## 4. Training

In [ ]:
import time

INIT_LR = 2e-7
MAX_LR = 2e-5

# Loss Function
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# Optimizer
steps_per_epoch = len(train_dataset) // BATCH_SIZE


clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=4 * steps_per_epoch
)

optimizer = tf.keras.optimizers.Adam(clr)
#optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

# Metrics

train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
train_loss_results = []

# Train step

@tf.function
def train_step(model, token_ids, masks, labels, lexicon, hash_lexicon, emoji):
    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask = masks, training=True, lexicon = lexicon, hash_lexicon=hash_lexicon, emoji=emoji)
        loss_value = loss_fn(labels, predictions)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_loss(loss_value)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])


# Validate step

@tf.function
def validation_step(model, token_ids, masks, labels, lexicon, hash_lexicon, emoji):
    predictions = model(token_ids, attention_mask=masks, training=False, lexicon=lexicon, hash_lexicon=hash_lexicon, emoji=emoji)
    v_loss = loss_fn(labels, predictions)
    validation_loss(v_loss)

    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

# Train loop


def train(model, train_dataset, val_dataset, epochs):
  for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))
    start_time = time.time()

    for step, (token_ids, masks, labels, lexicon, hash_lexicon, emoji) in enumerate(tqdm(train_dataset)):
      train_step(model, token_ids, masks, labels, lexicon, hash_lexicon, emoji)
      if step % 100 == 0:
        train_loss_results.append(train_loss.result())
        print(f'\nTrain Step: {step}, Loss: {train_loss.result()}')
        for i, label_name in enumerate(label_cols):
          print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
          train_auc_metrics[i].reset_states()

    for i, (token_ids, masks, labels, lexicon, hash_lexicon, emoji) in enumerate(tqdm(val_dataset)):
            validation_step(model, token_ids, masks, labels, lexicon, hash_lexicon, emoji)

    # Display metrics at the end of each epoch.
    print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start_time}\n')


    for i, label_name in enumerate(label_cols):
      print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
      validation_auc_metrics[i].reset_states()

    print('\n')

train(model, train_dataset, validation_dataset, epochs=3)


Start of epoch 1


  0%|          | 0/1449 [00:00<?, ?it/s]

(16, 1024)
Tensor("emoji:0", shape=(16, 8), dtype=float32)
(16, 8)
(16, 1050)

Train Step: 0, Loss: 0.7471149563789368
anger roc_auc 0.6583333015441895
anticipation roc_auc 0.0
disgust roc_auc 0.5333333611488342
fear roc_auc 0.46666663885116577
joy roc_auc 0.8671875
love roc_auc 0.8035714030265808
optimism roc_auc 0.34375
pessimism roc_auc 0.4871794879436493
sadness roc_auc 0.4365079402923584
surprise roc_auc 0.29999998211860657
trust roc_auc 0.0

Train Step: 100, Loss: 0.6004728674888611
anger roc_auc 0.500566840171814
anticipation roc_auc 0.5058721303939819
disgust roc_auc 0.5286256074905396
fear roc_auc 0.47953975200653076
joy roc_auc 0.5777170658111572
love roc_auc 0.5266552567481995
optimism roc_auc 0.4696265757083893
pessimism roc_auc 0.48409968614578247
sadness roc_auc 0.5077286958694458
surprise roc_auc 0.5088157057762146
trust roc_auc 0.4847802221775055

Train Step: 200, Loss: 0.5302143096923828
anger roc_auc 0.6575085520744324
anticipation roc_auc 0.5017145872116089
disgust r

  0%|          | 0/167 [00:00<?, ?it/s]

(2, 1024)
Tensor("emoji:0", shape=(2, 8), dtype=float32)
(2, 8)
(2, 1050)

Epoch 1, Validation Loss: 0.22708958387374878, Time: 712.9749689102173

anger roc_auc 0.9591218829154968
anticipation roc_auc 0.8285479545593262
disgust roc_auc 0.9535214304924011
fear roc_auc 0.9757160544395447
joy roc_auc 0.9765108823776245
love roc_auc 0.9412241578102112
optimism roc_auc 0.9380313158035278
pessimism roc_auc 0.8714150190353394
sadness roc_auc 0.9322147965431213
surprise roc_auc 0.9004421234130859
trust roc_auc 0.8796886205673218



Start of epoch 2


...


Epoch 2, Validation Loss: 0.21286353468894958, Time: 646.1672470569611

anger roc_auc 0.9745098352432251
anticipation roc_auc 0.8672816753387451
disgust roc_auc 0.9626259803771973
fear roc_auc 0.9822713136672974
joy roc_auc 0.9847860336303711
love roc_auc 0.9612230658531189
optimism roc_auc 0.9506731629371643
pessimism roc_auc 0.8937234282493591
sadness roc_auc 0.9439005255699158
surprise roc_auc 0.9293361902236938
trust roc_auc 0.9073445796966553



Start of epoch 3


  0%|          | 0/1449 [00:00<?, ?it/s]


Train Step: 0, Loss: 0.2725018858909607
anger roc_auc 0.9753366708755493
anticipation roc_auc 0.8522686958312988
disgust roc_auc 0.9419907331466675
fear roc_auc 0.9649404883384705
joy roc_auc 0.9704942107200623
love roc_auc 0.9734985828399658
optimism roc_auc 0.9520566463470459
pessimism roc_auc 0.8955134153366089
sadness roc_auc 0.9299622178077698
surprise roc_auc 0.897352933883667
trust roc_auc 0.9322196841239929

Train Step: 100, Loss: 0.27046895027160645
anger roc_auc 0.9776913523674011
anticipation roc_auc 0.8650206327438354
disgust roc_auc 0.946215808391571
fear roc_auc 0.9708379507064819
joy roc_auc 0.9759880900382996
love roc_auc 0.9582774043083191
optimism roc_auc 0.9481241703033447
pessimism roc_auc 0.8901145458221436
sadness roc_auc 0.9219517111778259
surprise roc_auc 0.9203723669052124
trust roc_auc 0.9195197820663452

Train Step: 200, Loss: 0.268367201089859
anger roc_auc 0.972843587398529
anticipation roc_auc 0.8652724623680115
disgust roc_auc 0.949846088886261
fear roc_

  0%|          | 0/167 [00:00<?, ?it/s]


Epoch 3, Validation Loss: 0.20545868575572968, Time: 646.4085493087769

anger roc_auc 0.9778119921684265
anticipation roc_auc 0.8753592371940613
disgust roc_auc 0.9634153842926025
fear roc_auc 0.983845591545105
joy roc_auc 0.9853804111480713
love roc_auc 0.9604231715202332
optimism roc_auc 0.9550999402999878
pessimism roc_auc 0.9076724052429199
sadness roc_auc 0.9528979659080505
surprise roc_auc 0.9378584027290344
trust roc_auc 0.9029229283332825




In [ ]:
model.save_weights("/content/drive/MyDrive/SemEval/multi_emotion_classification_lexicon.h5")

## 5. Evaluate

In [ ]:
df_result = test_set.copy(deep=True)
df_result = df_result.drop_duplicates('Tweet', keep='first')

df_result[label_cols] = 0.5
df_result.set_index('Tweet',inplace=True)

df_result.head()
test_set.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@USER @USER Dont worry Indian army is on its w...,True,True,False,False,False,False,True,False,False,False,True
1,"Academy of Sciences , eschews the normally sob...",False,False,True,False,False,False,False,False,False,False,False
2,I blew that opportunity - __ - #mad,True,False,True,False,False,False,False,False,True,False,False
3,This time in 2 weeks I will be 30 ... :sad_but...,False,False,False,False,True,False,False,False,True,False,False
4,#Deppression is real . Partners w / #depressed...,False,False,False,True,False,False,False,False,True,False,False


In [ ]:
model.load_weights('/content/drive/MyDrive/SemEval/multi_emotion_classification_lexicon.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, classification_report, hamming_loss, f1_score, accuracy_score, jaccard_score

#test_steps = len(test_set) // BATCH_SIZE

test_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

for i, (token_ids, masks, lexicon, hash_lexicon, emoji) in enumerate(tqdm(test_dataset)):
    labels = test_labels[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
    labels = tf.dtypes.cast(labels, tf.float32)
    
    sample_ids = test_set.iloc[i*BATCH_SIZE:(i+1)*BATCH_SIZE]['Tweet']
    predictions = model(token_ids, attention_mask=masks, lexicon=lexicon, hash_lexicon=hash_lexicon, emoji=emoji).numpy()

    df_result.loc[sample_ids, label_cols] = predictions

    for i, auc in enumerate(test_auc_metrics):
      auc.update_state(labels[:,i], predictions[:,i])

  0%|          | 0/204 [00:00<?, ?it/s]

(11, 1024)
Tensor("emoji:0", shape=(11, 8), dtype=float32)
(11, 8)
(11, 1050)


In [ ]:
for i, label_name in enumerate(label_cols):
    print(f"{label_name} roc_auc {test_auc_metrics[i].result()}")
    test_auc_metrics[i].reset_states()

anger roc_auc 0.939899206161499
anticipation roc_auc 0.7655539512634277
disgust roc_auc 0.9171308875083923
fear roc_auc 0.9446130394935608
joy roc_auc 0.9518322348594666
love roc_auc 0.9160152673721313
optimism roc_auc 0.9052020311355591
pessimism roc_auc 0.843501627445221
sadness roc_auc 0.9148129224777222
surprise roc_auc 0.7917962670326233
trust roc_auc 0.8117600679397583


In [ ]:
y_true = test_labels

best_thresh = 0
best_acc = 0
for thresh in np.arange(0.1, 1, 0.01):
  y_pred = df_result[label_cols].values
  y_pred = np.array([[1 if i > thresh else 0 for i in j] for j in y_pred])
  acc = jaccard_score(y_true, y_pred, average='samples')
  
  if acc > best_acc:
    best_thresh = thresh
    best_acc = acc
    
print(best_thresh)

In [ ]:
from sklearn.metrics import accuracy_score, jaccard_score

y_true = test_labels
y_pred = df_result[label_cols].values

thresh = best_thresh
y_pred = np.array([[1 if i > thresh else 0 for i in j] for j in y_pred])


print('Micro F1',f1_score(y_true, y_pred,average='micro'))
print('Macro F1',f1_score(y_true, y_pred,average='macro'))
print('Accuracy', jaccard_score(y_true, y_pred, average='samples'))

Micro F1 0.7316110969712395
Macro F1 0.5897774552944518
Accuracy 0.617262817983898


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(classification_report(y_true, y_pred))
#label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1101
           1       0.45      0.17      0.25       425
           2       0.78      0.76      0.77      1099
           3       0.79      0.68      0.73       485
           4       0.87      0.86      0.87      1442
           5       0.70      0.58      0.64       516
           6       0.76      0.73      0.75      1143
           7       0.56      0.24      0.34       375
           8       0.80      0.70      0.75       960
           9       0.43      0.19      0.27       170
          10       0.27      0.06      0.10       153

   micro avg       0.78      0.67      0.72      7869
   macro avg       0.66      0.53      0.57      7869
weighted avg       0.75      0.67      0.70      7869
 samples avg       0.77      0.69      0.70      7869



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
